In [1]:
'''
    Opening notebook remotely on MacOS --> jupyter-notebook --no-browser --port=9090
'''

'\n    Opening notebook remotely on MacOS --> jupyter-notebook --no-browser --port=9090\n'

In [7]:
from sklearn.metrics import classification_report
from keras.models import load_model
import numpy as np
from sklearn.metrics import roc_curve, auc
import pandas as pd
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [4]:
orig_model_name = "orig_example.h5"
aug_model_name = "aug_example.h5"
orig_model = load_model(orig_model_name)
aug_model = load_model(aug_model_name)

OSError: No file or directory found at orig_example.h5

In [8]:
shared_dir = "/s/bach/a/class/cs435/cs435a/"
image_dir = "CS435_Plant_Data_v2_augment-v2/2023-11-15-231056/"

orig_test = "splits/original/orig_test.csv"
aug_test = "splits/augmented/aug_test.csv"

orig_test_path = shared_dir+image_dir+orig_test
aug_test_path = shared_dir+image_dir+aug_test

In [9]:
orig_test_df = pd.read_csv(orig_test_path)
aug_test_df = pd.read_csv(aug_test_path)

datagen = ImageDataGenerator(rescale=1./255) #normalize from [0,255] to [0,1]
orig_test_generator = datagen.flow_from_dataframe(
    dataframe=orig_test_df, 
    x_col='path_to_shared',
    y_col='class_name',  
    target_size=(224, 224),
    batch_size=4,
    class_mode='categorical', 
    shuffle=False
)

aug_test_generator = datagen.flow_from_dataframe(
    dataframe=aug_test_df, 
    x_col='path_to_shared',
    y_col='class_name',  
    target_size=(224, 224),
    batch_size=1,
    class_mode='categorical', 
    shuffle=False
)

Found 7166 validated image filenames belonging to 31 classes.
Found 21580 validated image filenames belonging to 31 classes.


In [10]:
# Basic metrics: accuracy, precision, recall, f1-score
def evaluate_model_metrics(model, test_generator):
    accuracy = model.evaluate(test_generator)[1]
    print(f"Test Accuracy: {evaluation[1]}")

    predictions = model.predict(test_generator)
    predicted_classes = [np.argmax(pred) for pred in predictions]

    true_classes = test_generator.classes

    class_labels = list(test_generator.class_indices.keys())

    report = classification_report(true_classes, predicted_classes, target_names=class_labels, output_dict=True)
    return report

In [11]:
# More advanced metrics: Confusion Matrix
def create_model_confusion_matrix(model, test_generator, model_name):
    predictions = model.predict(test_generator)
    
    predicted_classes = np.argmax(predictions, axis=1)

    label_map = {label: index for index, label in enumerate(test_generator.class_indices)}

    true_classes = test_df['class_name'].map(label_map).values

    conf_matrix = confusion_matrix(true_classes, predicted_classes)

    plt.figure(figsize=(20, 20))
    sns.heatmap(conf_matrix, annot=True, fmt='g')
    plt.xlabel('Predicted labels')
    plt.ylabel('True labels')
    plt.title('Confusion Matrix')

    plt.savefig(model_name+'_aug_confusion_matrix.png')
    plt.show()

In [12]:
print("*** EVALUATING MODEL TRAINED ON ORIGINAL DATA ***")
orig_orig_test_report = evaluate_model_metrics(orig_model, orig_test_generator)
orig_aug_test_report = evaluate_model_metrics(orig_model, aug_test_generator)

print("*** EVALUATIONS ON ORIGINAL TEST DATA ***")
orig_orig_df = pd.Dataframe(orig_orig_test_report).transpose()
display(orig_orig_df)

print("*** EVALUATIONS ON AUGMENTED TEST DATA ***")
orig_aug_df = pd.Dataframe(orig_aug_test_report).transpose()
display(orig_aug_df)

*** EVALUATING MODEL TRAINED ON ORIGINAL DATA ***


NameError: name 'orig_model' is not defined

In [13]:
print("*** EVALUATING MODEL TRAINED ON ORIGINAL DATA ***")
aug_orig_test_report = evaluate_model_metrics(aug_model, orig_test_generator)
aug_aug_test_report = evaluate_model_metrics(aug_model, aug_test_generator)

print("*** EVALUATIONS ON ORIGINAL TEST DATA ***")
aug_orig_df = pd.Dataframe(aug_orig_test_report).transpose()
display(aug_orig_df)

print("*** EVALUATIONS ON AUGMENTED TEST DATA ***")
aug_aug_df = pd.Dataframe(aug_aug_test_report).transpose()
display(aug_aug_df)

*** EVALUATING MODEL TRAINED ON ORIGINAL DATA ***


NameError: name 'aug_model' is not defined